# Google OAuth Testing

This notebook helps you test Google OAuth authentication endpoints.

## Endpoints Covered:
- `/accounts/google/login/` - Initiate Google OAuth flow
- `/accounts/google/login/callback/` - OAuth callback
- `/accounts/google/login/token/` - Login with Google token

## Prerequisites:
1. Google OAuth credentials must be configured in `.env`
2. SocialApp must be created in Django admin
3. Redirect URIs must be configured in Google Cloud Console

## Setup

In [ ]:
import requests
from pprint import pprint

BASE_URL = "http://localhost:8000"

def print_response(response):
    """Pretty print response"""
    print(f"Status Code: {response.status_code}")
    print(f"\nResponse Body:")
    try:
        pprint(response.json())
    except:
        print(response.text)

print("✅ Setup complete!")

## 1. Check Google OAuth Configuration

Verify that Google OAuth is properly configured

In [ ]:
# Check if Google login URL is accessible
response = requests.get(f"{BASE_URL}/accounts/google/login/")

print(f"Status Code: {response.status_code}")

if response.status_code == 200:
    print("✅ Google OAuth endpoint is accessible!")
    print("\nYou can open this URL in a browser:")
    print(f"{BASE_URL}/accounts/google/login/")
else:
    print("❌ Google OAuth might not be configured properly")
    print_response(response)

## 2. Get Google OAuth Authorization URL

This will show you the redirect URL for Google OAuth

In [ ]:
# Make a request without following redirects
response = requests.get(
    f"{BASE_URL}/accounts/google/login/",
    allow_redirects=False
)

if response.status_code in [301, 302]:
    redirect_url = response.headers.get('Location')
    print("✅ Google OAuth is configured!")
    print(f"\nRedirect URL: {redirect_url[:100]}...")
else:
    print_response(response)

## 3. Login with Google ID Token

If you have a Google ID token, you can use it to authenticate

In [ ]:
# You would get this token from Google OAuth flow in a real frontend app
google_token_data = {
    "id_token": "YOUR_GOOGLE_ID_TOKEN_HERE",
    "access_token": "YOUR_GOOGLE_ACCESS_TOKEN_HERE"
}

response = requests.post(
    f"{BASE_URL}/accounts/google/login/token/",
    json=google_token_data
)

print_response(response)

## 4. Manual OAuth Flow Simulation

To complete the OAuth flow manually:

1. Open browser and visit: `http://localhost:8000/accounts/google/login/`
2. Login with your Google account
3. You'll be redirected back with an authorization code
4. The backend will exchange the code for tokens automatically

After successful login, you'll receive JWT tokens.

## Configuration Check

Check if Google OAuth is properly set up in the database

In [ ]:
# You need to be logged in as admin for this
print("To check Google OAuth configuration:")
print("1. Login to admin panel: http://localhost:8000/admin/")
print("2. Navigate to: Social Accounts > Social applications")
print("3. Verify Google provider is configured with:")
print("   - Provider: Google")
print("   - Client ID: Your Google Client ID")
print("   - Secret key: Your Google Client Secret")
print("   - Sites: Your site (example.com or localhost)")

## Testing OAuth Flow with Frontend

Here's a sample JavaScript code for your frontend:

In [ ]:
print("""
// Sample React/JavaScript code for Google OAuth

// 1. Install Google Sign-In library
npm install @react-oauth/google

// 2. Use GoogleLogin component
import { GoogleLogin } from '@react-oauth/google';

const handleGoogleSuccess = async (credentialResponse) => {
  const { credential } = credentialResponse;
  
  // Send to your Django backend
  const response = await fetch('http://localhost:8000/accounts/google/login/token/', {
    method: 'POST',
    headers: {
      'Content-Type': 'application/json',
    },
    body: JSON.stringify({
      id_token: credential,
    }),
  });
  
  const data = await response.json();
  console.log('JWT Tokens:', data);
  // Store access and refresh tokens
};

<GoogleLogin
  onSuccess={handleGoogleSuccess}
  onError={() => console.log('Login Failed')}
/>
""")